In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape(urls, output_file):
    all_data = []

    for url in urls:
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        headings = soup.find_all('h2')
        data = []
        for heading in headings:
            content = []
            element = heading.find_next_sibling()

            while element is not None and not (element.name and element.name.startswith('h')):
                text = element.get_text(strip=True)
                if text:
                    content.append(text)
                element = element.find_next_sibling()

            data.append({
                'URL': url,
                'Heading': heading.get_text(strip=True),
                'Content': ' '.join(content)
            })
        
        all_data.extend(data)

    df = pd.DataFrame(all_data)
    df.to_csv(output_file, index=False)
    print(f'Data written to {output_file}')

urls = [
    'https://www.cdc.gov/poxvirus/mpox/if-sick/transmission.html',
    'https://www.cdc.gov/poxvirus/mpox/prevention/sexual-health.html',
    'https://www.cdc.gov/poxvirus/mpox/symptoms/index.html',
    'https://www.cdc.gov/poxvirus/mpox/vaccines/index.html',
    'https://www.who.int/news-room/fact-sheets/detail/monkeypox'
]

output_file = 'scraped_data.csv'
scrape(urls, output_file)

Data written to scraped_data.csv
